In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder



2023-05-30 16:34:27.431458: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 16:34:28.322168: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/panda/test_ws/devel/lib:/home/panda/ws_moveit/devel/lib:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu
2023-05-30 16:34:28.322308: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /ho

In [2]:
data_dir = '/home/panda/model_data/cnn_data'

# Step 1: Load and preprocess the data

# Initialize empty lists to store the data and labels
data = []
labels = []
recorded_array=[]
count=0
class_names = ['push', 'pull', 'up', 'down', 'stop','rotate_cw','rotate_ccw','rotate_up','rotate_down']
# Iterate over each subdirectory (class folder) in the data directory
for i,class_name in enumerate    (class_names):
    
    class_path = os.path.join(data_dir, class_names[i])
    if os.path.isdir(class_path):
        # Iterate over each sample file in the class folder
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            # count+=1
            if file_name.endswith('.npy'):
                  
                # Load the recorded array from the file
                recorded_array = np.load(file_path)
                # Check if the shape of the recorded array matches the expected shape
                expected_shape = (210, )  # Adjust the shape according to your data
                if recorded_array.shape == expected_shape:
                    # Append the recorded array to the data list
                    data.append(recorded_array.flatten())
                    # Append the corresponding label to the labels list
                    label = class_names.index(class_name)
                    labels.append(label)
                    count+=1
                    recorded_array=[]
                else:
                    continue
    print(f'data points in class {class_name} are {count}') 
    count=0         
# Convert the data and labels lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Step 2: Preprocess the data

# Perform any required preprocessing steps, such as scaling or normalization
# Here, we will use StandardScaler to scale the data
print(data.shape)
print (labels.shape)
print(len(class_names))
print(count)

data points in class push are 2000
data points in class pull are 2000
data points in class up are 2000
data points in class down are 2000
data points in class stop are 2000
data points in class rotate_cw are 2000
data points in class rotate_ccw are 2000
data points in class rotate_up are 2000
data points in class rotate_down are 2000
(18000, 210)
(18000,)
9
0


In [81]:
data_dir = '/home/rehan/catkin_ws/src/panda_research/hand_classifier/data/cnn_data'

# Step 1: Load and preprocess the data

# Initialize empty lists to store the data and labels
data = []
labels = []
recorded_array=[]
count=0
class_names = ['push', 'pull', 'up', 'down', 'stop','rotate_cw','rotate_ccw']
class_counts = []  # Initialize a list to store the counts of valid samples per class

# Iterate over each subdirectory (class folder) in the data directory
for i, class_name in enumerate(class_names):
    class_path = os.path.join(data_dir, class_name)
    class_count = 0  # Initialize the count for the current class

    if os.path.isdir(class_path):
        # Iterate over each sample file in the class folder
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)

            if file_name.endswith('.npy'):
                # Load the recorded array from the file
                recorded_array = np.load(file_path)
                expected_shape = (210,)  # Adjust the shape according to your data

                if recorded_array.shape == expected_shape:
                    # Append the recorded array to the data list
                    data.append(recorded_array.flatten())
                    # Append the corresponding label to the labels list
                    label = class_names.index(class_name)
                    labels.append(label)
                    class_count += 1  # Increment the count for the current class

    class_counts.append(class_count)  # Store the count for the current class

# Convert the data and labels lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Print the counts for each class
for i, class_name in enumerate(class_names):
    print(f"{class_name}: {class_counts[i]}")

push: 2000
pull: 2000
up: 2000
down: 2000
stop: 2000
rotate_cw: 2000
rotate_ccw: 2000


In [3]:
# Split the data and labels into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [4]:
scaler = StandardScaler()
normalizer = MinMaxScaler(feature_range=(0, 1))

data_scaled = scaler.fit_transform(data)
data_normalized = normalizer.fit_transform(data_scaled)

train_data, val_data, train_labels, val_labels = train_test_split(data_normalized, labels, test_size=0.2, random_state=42)

print(train_data.shape)
print(val_data.shape)
print(train_labels.shape)
print(val_labels.shape)

(14400, 210)
(3600, 210)
(14400,)
(3600,)


In [5]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(210,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(class_names), activation='softmax'))


2023-05-30 16:36:07.876137: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 16:36:09.024234: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6757 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:65:00.0, compute capability: 6.1


In [6]:

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [7]:
# Train the model
model.fit(train_data, train_labels, batch_size=32, epochs=100, validation_data=(val_data, val_labels))

Epoch 1/100
450/450 [==============================] - 4s 6ms/step - loss: 1.3938 - accuracy: 0.5088 - val_loss: 0.3063 - val_accuracy: 0.9725
Epoch 2/100
450/450 [==============================] - 2s 5ms/step - loss: 0.5930 - accuracy: 0.7877 - val_loss: 0.0707 - val_accuracy: 0.9947
Epoch 3/100
450/450 [==============================] - 2s 5ms/step - loss: 0.4275 - accuracy: 0.8431 - val_loss: 0.0555 - val_accuracy: 0.9950
Epoch 4/100
450/450 [==============================] - 3s 6ms/step - loss: 0.3591 - accuracy: 0.8724 - val_loss: 0.0369 - val_accuracy: 0.9953
Epoch 5/100
450/450 [==============================] - 2s 5ms/step - loss: 0.3124 - accuracy: 0.8856 - val_loss: 0.0230 - val_accuracy: 0.9967
Epoch 6/100
450/450 [==============================] - 2s 6ms/step - loss: 0.2867 - accuracy: 0.8933 - val_loss: 0.0197 - val_accuracy: 0.9961
Epoch 7/100
450/450 [==============================] - 2s 5ms/step - loss: 0.2582 - accuracy: 0.9078 - val_loss: 0.0153 - val_accuracy: 0.9981

In [8]:
# Get the 9predicted labels from the model
predicted_labels = model.predict(val_data)
predictions = np.around(model.predict(val_data))
# print(predictions)
# Decode the predicted labels
print((predicted_labels.shape))
for i,prediction in enumerate (predictions):
    # print(np.argmax(predictions[i][0]))
    label=np.argmax(predictions)
    print(class_names[label])
# print(prediction_labels[0][np.argmax(prediction)])
print(class_names)

113/113 [==============================] - 0s 2ms/step
(3600, 9)
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull


In [9]:
import pickle
# Save the model
model.save('/home/panda/model_data/model1/keras_model.h5')

# Save the labels to a text file
with open('/home/panda/model_data/model1/labels.txt', 'w') as f:
    for label in class_names:
        f.write(label + '\n')
# Saving the scaler and normalizer objects
with open('/home/panda/model_data/model1/scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

with open('/home/panda/model_data/model1/normalizer.pkl', 'wb') as normalizer_file:
    pickle.dump(normalizer, normalizer_file)